In [427]:
import pandas as pd
import re
import json
import time

# Data Read-in

In [482]:
# Display all content
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', 100)
pd.set_option("display.max_colwidth", 100)
pd.set_option('display.width', 100)

In [429]:
data = pd.read_csv('labeled_data.csv')

In [430]:
# Just a bunch of cleaning stuff for irrelevant data
data = data[['item_name', 'title', 'content_','label']]
data.columns = ['tag', 'title', 'content', 'label']

In [175]:
%cd /Users/yantong/Desktop/NLP\ for\ Risk/Code/

/Users/yantong/Desktop/NLP for Risk/Code


In [176]:
def flatten(obj, name = ''):
    if isinstance(obj, dict):
        for key, value in obj.items():
            flatten(obj[key], key)
    else:
        flattened_dict[name] = [obj] 


In [340]:
toplevelstring = ''
original_data = pd.DataFrame()
with open('dc_wx_article.json',encoding='UTF-8') as f:   
    i = 1
    for line in f:
        if (not '}}}' in toplevelstring.replace('\n', '').replace('\s+','')) & (i<=60000) :
            #toplevelstring = toplevelstring + line
            toplevelstring = line
            i = i + 1
            data = json.loads(toplevelstring)
            flattened_dict = {}
            flatten(data)
            df = pd.DataFrame.from_dict(flattened_dict)
            original_data = pd.concat([original_data,df],axis = 0)
        else:
            break

In [341]:
text_data = original_data.drop(['_index'],axis = 1)

In [431]:
data = text_data.copy()

In [432]:
data = data.reset_index()

# Text Cleaning

In [433]:
# Remove Whitespaces and Newlines
pattern = r'[\n ]'
data['content'] = pd.Series([re.sub(pattern, '', text) for text in data.content])

In [434]:
def cleaned_content(row):
    row_content = row['content']
    row_title = row['title']
    row_sourceName = row['sourceName']
    source_name_len = len(row_sourceName)
    wxh_idx = row_content.find('微信号')
    gnjs_idx = row_content.find('功能介绍')
    global row_content_cleaned
    if (gnjs_idx>wxh_idx>0):
        rmv_h = wxh_idx - 2*source_name_len
        rmv_t = gnjs_idx + 4
        row_content_cleaned = row_content[:rmv_h]+'。'+row_content[rmv_t:]
    
    kyk_idx_h = row_content_cleaned.find('看一看')
    kyk_idx_t = row_content_cleaned.rfind('看一看')
    if ((kyk_idx_t - kyk_idx_h <200) & (kyk_idx_t>0)):
        row_content_cleaned = row_content_cleaned[:kyk_idx_h]
        
    yd_idx = row_content_cleaned.rfind('阅读',kyk_idx_h-20)
    if ((yd_idx>0)& (kyk_idx_t>0)):
        row_content_cleaned = row_content_cleaned[:yd_idx]
        
    row['cleaned_content'] = row_content_cleaned    
    return row


In [435]:
data.shape

(7000, 19)

In [436]:
data = data.apply(lambda row: cleaned_content(row),axis = 1)

In [437]:
# check
data['content_count'] = data['content'].apply(lambda row: len(row))
data['cleaned_content_count'] = data['cleaned_content'].apply(lambda row: len(row))

In [438]:
max(data.content_count - data.cleaned_content_count)
# 最多去除193个词

265

# Split Text

In [439]:
# Split text into blocks, by 500 characters, separated by period(, or comma), or whatever
# Return a list of strings

def split_text(text, size = 500):
    
    # Argument Restrictions
    if type(text) != str:
        raise ValueError("Input text must be a string!")
    
    if size < 100:
        raise ValueError("Split size must be g.o.e 100!")
    elif size > 600:
        raise ValueError("Split size must be l.o.e 600!")
    
    # Initialize
    length = len(text)
    text_block = []
    i, j = 0, size
    
    while j < length:
        while text[j - 1] != '。':
            j -= 1
            if j == i:
                j = i + size
                while text[j - 1] != '，':
                    j -= 1
                    if j == i:
                        j = i + size
                        break
                break
        text_block.append(text[i:j])
        i = j
        j = i + size
    text_block.append(text[i:length])
    
    return text_block

In [440]:
data['split'] = pd.Series([split_text(text) for text in data.cleaned_content])

In [441]:
# Remove Stopwords
# stopwords = [line.strip() for line in open('stopwords.txt', encoding = 'UTF-8').readlines()]

In [442]:
# def move_stopwords(wordlist, stopwords):
#     output = [word for word in wordlist if word not in stopwords]
#     return output

In [443]:
sum(data.split.apply(lambda row: len(row)))

41790

# Baidu

In [444]:
# encoding:utf-8
import requests 
from aip import AipNlp

# client_id 为官网获取的AK， client_secret 为官网获取的SK
host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id=WhgBKh94vkLS7IEN01GmynsA&client_secret=XWYOMfoCadaQnTEZDgCARpwoq41eNM4y'
response = requests.get(host)
if response:
    print(response.json())

client = AipNlp('23189963', 'HI5W90NErK56kv8sItGydAkg', 'KkpstXf5HpQ45wHyBk1Lz0CK4vhN20V9')


{'refresh_token': '25.1858dcf74f830305ee2dc0aabe5f135e.315360000.1924498804.282335-23257388', 'expires_in': 2592000, 'session_key': '9mzdXvFsqYeYU57vopTnYO77OpHWeF9VB7OrC/z9XHWX2sIHYbztHpHyJl0GdqEMbWEBDv+znxO7WDYSj3GMFCkM7qSQkA==', 'access_token': '24.e401c35e8c097a67e4c1eeceb6c0d141.2592000.1611730804.282335-23257388', 'scope': 'public nlp_simnet nlp_wordemb nlp_comtag nlp_dnnlm_cn brain_nlp_lexer brain_all_scope brain_nlp_comment_tag brain_nlp_dnnlm_cn brain_nlp_word_emb_vec brain_nlp_word_emb_sim brain_nlp_sentiment_classify brain_nlp_simnet brain_nlp_depparser brain_nlp_wordembedding brain_nlp_dnnlm_cn_legacy brain_nlp_simnet_legacy brain_nlp_comment_tag_legacy brain_nlp_lexer_custom brain_nlp_keyword brain_nlp_topic brain_nlp_ecnet brain_nlp_emotion brain_nlp_comment_tag_custom brain_nlp_news_summary brain_nlp_sentiment_classify_custom brain_nlp_address wise_adapt lebo_resource_base lightservice_public hetu_basic lightcms_map_poi kaidian_kaidian ApsMisTest_Test权限 vis-classify_flow

In [445]:
# Convert utf-8 to gbk

def killAnUnseen(s):
    try:
        s.encode('gbk')
        return s
    except UnicodeEncodeError as err:
        mode = re.findall(r'position ([0-9]*): illegal multibyte sequence', str(err))
        news = s.replace(s[int(mode[0])], "", 1)
        return killAnUnseen(news)

In [446]:
# Get both baidu segmentation and sentiment

def baidu(text_list):
    seg_list = []
    prob_list = []
    length = len(text_list)
    i = 0
    
    # 2 operations per loop
    while i + 1 < length:
        prob_list.append(baidu_prob(text_list[i]))
        prob_list.append(baidu_prob(text_list[i+1]))
        seg_list.append(baidu_seg(text_list[i]))
        seg_list.append(baidu_seg(text_list[i+1]))
        i += 2
        time.sleep(1)
    
    if i == length - 1:
        prob_list.append(baidu_prob(text_list[i]))
        seg_list.append(baidu_seg(text_list[i]))
        time.sleep(1)
        
    return seg_list, prob_list


def baidu_seg(text):
    result_list = []
    result = client.lexer( killAnUnseen(text))
    while len(result) == 2:
        time.sleep(1)
        result = client.lexer( killAnUnseen(text))
    for item in result['items']: 
        result_list.append(item['item'])
    result_list = move_stopwords(result_list, stopwords)
    
    return result_list


def baidu_prob(text):
    result = client.sentimentClassify( killAnUnseen(text) )
    while len(result) == 2:
        time.sleep(1)
        result = client.sentimentClassify( killAnUnseen(text) )
    return result['items'][0]['negative_prob']

In [447]:
data['baidu_seg'] = ''
data['baidu_prob'] = ''

In [448]:
data = data[['cleaned_content', 'split', 'baidu_seg', 'baidu_prob']]

In [449]:
data = data.reset_index()

In [450]:
def write_value(i):
    length = len(data.index)
    
    while i < length:
        seg, prob = baidu(data.at[i, 'split'])
        data.at[i, 'baidu_seg'] = seg
        data.at[i, 'baidu_prob'] = prob
        print(str(i))
        i += 1
        
    print("Finish All!")

In [485]:
write_value(6740)

6740
6741
6742
6743
6744
6745
6746
6747
6748
6749
6750
6751
6752
6753
6754
6755
6756
6757
6758
6759
6760
6761
6762
6763
6764
6765
6766
6767
6768
6769
6770
6771
6772
6773
6774
6775
6776
6777
6778
6779
6780
6781
6782
6783
6784
6785
6786
6787
6788
6789
6790
6791
6792
6793
6794
6795
6796
6797
6798
6799
6800
6801
6802
6803
6804
6805
6806
6807
6808
6809
6810
6811
6812
6813
6814
6815
6816
6817
6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939


In [486]:
#data.to_csv('split.csv')

In [465]:
data.loc[3770,'baidu_seg']

[['钱',
  '去哪儿',
  '任泽平',
  '格上理财',
  '旗下',
  '专注',
  '私募',
  '行业',
  '微信公众号',
  '每天',
  '精选',
  '私募',
  '行业',
  '动态',
  '私募',
  '大佬',
  '观点',
  '私募',
  '研究',
  '报告',
  '分享',
  '涵盖',
  '宏观经济',
  '个股',
  '投资',
  '理念',
  '方面',
  '精华',
  '文章',
  '了解',
  '私募',
  '行业',
  '关注',
  '格上',
  '私募',
  '圈',
  '作者',
  '恒大研究院',
  '任泽平',
  '石玲玲',
  '王孟嫫',
  '来源',
  '泽平',
  '宏观',
  'ID',
  'zeping',
  'hongguan',
  '2016年底',
  '2017年初',
  '严',
  '监管',
  '金融',
  '杠杆',
  '拉开',
  '帷幕',
  '广义货币',
  '供应',
  '增速',
  'M2',
  '创',
  '历史',
  '新低',
  '影子银行',
  '快速',
  '收缩',
  '社会',
  '融资',
  '规模',
  '尤其',
  '表外融资',
  '大幅',
  '下降',
  '进',
  '引发',
  '资产',
  '价格',
  '调整',
  '中小企业',
  '融资难',
  '贵',
  '信用债',
  '风险',
  '股权',
  '融资',
  '爆仓',
  'P2P',
  '爆',
  '雷',
  '一系列',
  '多米诺骨牌效应',
  '本文',
  '旨在',
  '货币',
  '创造',
  '资产',
  '形成',
  '角度',
  '探究',
  '钱',
  '去哪儿',
  '过去',
  '超',
  '发',
  '钱',
  '好像',
  '一夜',
  '之间',
  '消失了',
  '摘要',
  '正确',
  '认识',
  '钱',
  '钱',
  '不是',
  '简单',
  '意义',
  '央行',
  '印钞',
  '央行',
  '商业银行',


In [466]:
data.loc[3770,:]

index                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   